In [ ]:
!pip uninstall -y tensorflow --quiet
!pip install ludwig
!pip install ludwig[llm]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 104.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 12.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 90.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 11.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=6ec6feb1a3e89185ded599bad817f43ae07d1aa0aa50a64c93ff0a69d4b92f27
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)

def clear_cache():
  if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [ ]:
import getpass
import locale; locale.getpreferredencoding = lambda: "UTF-8"
import logging
import os
import torch
import yaml

from ludwig.api import LudwigModel


os.environ["HUGGING_FACE_HUB_TOKEN"] = getpass.getpass("Token:")
assert os.environ["HUGGING_FACE_HUB_TOKEN"]

Token:··········


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import data_table; data_table.enable_dataframe_formatter()
import numpy as np; np.random.seed(123)
import pandas as pd

df = pd.read_parquet('/content/drive/MyDrive/train-00000-of-00001-dc051879a2b92047.parquet')

# We're going to create a new column called `split` where:
# 90% will be assigned a value of 0 -> train set
# 5% will be assigned a value of 1 -> validation set
# 5% will be assigned a value of 2 -> test set
# Calculate the number of rows for each split value

total_rows = len(df)
split_0_count = int(total_rows * 0.9)
split_1_count = int(total_rows * 0.05)
split_2_count = total_rows - split_0_count - split_1_count

# Create an array with split values based on the counts

split_values = np.concatenate([
    np.zeros(split_0_count),
    np.ones(split_1_count),
    np.full(split_2_count, 2)
])

# Shuffle the array to make sure that the training examples

np.random.shuffle(split_values)

# Add the 'split' column to the DataFrame

df['split'] = split_values
df['split'] = df['split'].astype(int)


In [ ]:
df.head(10)

,text,label,split
0,"effective, makes browsing smoother than a vr6 ...",1,0
1,worked really great and made different sites l...,1,0
2,i've only had this for a little while and i do...,1,0
3,it was great but recently stopped working. upd...,0,0
4,best ad blocker out there right now no other c...,1,0
5,completely impressed. this app works at its be...,1,0
6,can not delete it or uninstall it\n\npermissio...,0,0
7,love it save my data and unwanted add\n,1,0
8,"blocks even the nastiest pop-ups, but doesn't ...",1,1
9,"works good, but now let's youtube ads load now...",0,0


In [ ]:
column_mapping = {'text': 'instruction', 'value': 'items'}

In [ ]:
df = df.rename(columns=column_mapping)

In [ ]:
df

,instruction,label,split
0,"effective, makes browsing smoother than a vr6 ...",1,0
1,worked really great and made different sites l...,1,0
2,i've only had this for a little while and i do...,1,0
3,it was great but recently stopped working. upd...,0,0
4,best ad blocker out there right now no other c...,1,0
...,...,...,...
746,perfect i love it!\n,1,0
747,great for added protection!\n,1,0
748,very good\n,1,0
749,noscript comes to chrome.\n,1,0


In [ ]:
#Zero-shot inferencing to get the insight of how the base model works on our dataset before training

zero_shot_config = yaml.safe_load(
  """
  model_type: llm
  base_model: meta-llama/Llama-2-7b-hf

  input_features:
    - name: instruction
      type: text

  output_features:
    - name: label
      type: text

  prompt:
    template: >-
      Below is an instruction that contains the text or the sentence in the form of a tweet. Give the response in the form so as to find whether it is a positive or negative tweet.
      ### Instruction: {instruction}

      ### Response:

  generation:
    temperature: 0.5 # Temperature is used to control the randomness of predictions.
    max_new_tokens: 8

  preprocessing:
    split:
      type: fixed
    tokenize:
      type: word

  quantization:
    bits: 4
  """
)

model = LudwigModel(config=zero_shot_config, logging_level=logging.INFO)
results = model.train(dataset=df[:10])

INFO:ludwig.utils.print_utils:
INFO:ludwig.utils.print_utils:╒════════════════════════╕
INFO:ludwig.utils.print_utils:│ EXPERIMENT DESCRIPTION │
INFO:ludwig.utils.print_utils:╘════════════════════════╛
INFO:ludwig.utils.print_utils:
INFO:ludwig.api:╒══════════════════╤═════════════════════════════════════════════════════════════════════════════════════════╕
│ Experiment name  │ api_experiment                                                                          │
├──────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ Model name       │ run                                                                                     │
├──────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ Output directory │ /content/results/api_experiment_run                                                     │
├──────────────────┼─────────────────────────────────────────────────────────────────

INFO:ludwig.utils.tokenizers:Loaded HuggingFace implementation of meta-llama/Llama-2-7b-hf tokenizer
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
INFO:ludwig.features.text_feature:Max length of feature 'None': 141 (without start and stop symbols)
INFO:ludwig.features.text_feature:Setting max length using dataset: 143 (including start and stop symbols)
INFO:ludwig.features.text_feature:max sequence length is 143 for feature 'None'
INFO:ludwig.utils.tokenizers:Loaded HuggingFace implementation of meta-llama/Llama-2-7b-hf tokenizer
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
INFO:ludwig.features.text_feature:Max length of feature 'label': 3 (without start and stop symbols)
INFO:ludwig.features.text_feature:Setting max length using dataset: 5 (including start and stop symbols)
INFO:ludwig.features.te

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:ludwig.models.llm:Done.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
INFO:ludwig.utils.tokenizers:Loaded HuggingFace implementation of meta-llama/Llama-2-7b-hf tokenizer
INFO:ludwig.utils.print_utils:
INFO:ludwig.utils.print_utils:╒══════════╕
INFO:ludwig.utils.print_utils:│ TRAINING │
INFO:ludwig.utils.print_utils:╘══════════╛
INFO:ludwig.utils.print_utils:
INFO:ludwig.trainers.trainer_llm:
Running evaluation for step: 0, epoch: 0


Evaluation valid:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:ludwig.models.llm:For generating text, using: GenerationConfig {
  "max_length": 32,
  "max_new_tokens": 8,
  "temperature": 0.5
}

INFO:ludwig.models.llm:Decoded text inputs for the first example in batch: below is an instruction that contains the text or the sentence in the form of a tweet. give the response in the form so as to find whether it is a positive or negative tweet. ### instruction: blocks even the nastiest pop-ups, but doesn't interfere with site interaction. even seems like it speeds up streaming by keeping all the side ads, videos and pop-ups from slowing down computer. awesome extension! i would pay for it.

### response:
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
INFO:ludwig.models.llm:Decoded generated

Evaluation valid: 100%|██████████| 1/1 [00:06<00:00,  6.04s/it]

INFO:ludwig.trainers.trainer_llm:Evaluation took 6.6780s

INFO:ludwig.utils.metrics_printed_table:╒═══════════════════════╤═════════╤══════════════╤════════╕
│                       │ train   │   validation │ test   │
╞═══════════════════════╪═════════╪══════════════╪════════╡
│ bleu                  │         │       0.0000 │        │
├───────────────────────┼─────────┼──────────────┼────────┤
│ char_error_rate       │         │      15.0000 │        │
├───────────────────────┼─────────┼──────────────┼────────┤
│ loss                  │         │      10.3735 │        │
├───────────────────────┼─────────┼──────────────┼────────┤
│ next_token_perplexity │         │   31999.9727 │        │
├───────────────────────┼─────────┼──────────────┼────────┤
│ perplexity            │         │   32000.0039 │        │
├───────────────────────┼─────────┼──────────────┼────────┤
│ rouge1_fmeasure       │         │       0.0000 │        │
├───────────────────────┼─────────┼──────────────┼────────┤
│ 

In [ ]:
#Training the Llama 2 model on our custom Tweet Classification dataset

model = None
clear_cache()

qlora_fine_tuning_config = yaml.safe_load(
"""
model_type: llm
base_model: meta-llama/Llama-2-7b-hf

input_features:
  - name: instruction
    type: text

output_features:
  - name: label
    type: text

prompt:
  template: >-
   Below is the text or the sentence in the form of a tweet. Detect the sentiment of the tweet and give the response in the form so as to find whether it is a positive or negative tweet. In the given training examples 0 indicates negative and 1 indicates positive.

    ### Instruction: {instruction}

    ### Response:

generation:
  temperature: 0.3
  max_new_tokens: 512


adapter:
  type: lora
  lora_activation: relu
  lora_type: lora_highway

quantization:
  bits: 4

preprocessing:
  global_max_sequence_length: 512
  tokenize:
      type: word
  stop_words:
    type: none
  lemmatize: True
  split:
    type: fixed
    probabilities:
    - 1
    - 0
    - 0

trainer:
  type: finetune
  train_steps: 20
  epochs: 2
  batch_size: 1
  eval_batch_size: 2
  gradient_accumulation_steps: 8
  learning_rate: 0.0005
  learning_rate_scheduler:
    type: cosine_annealing
  adam_beta1: 0.9
  adam_beta2: 0.99
  epsilon: 1e-8
  warmup_fraction: 0.06
  weight_decay: 1e-5
"""
)

model = LudwigModel(config=qlora_fine_tuning_config, logging_level=logging.INFO)
results = model.train(dataset=df)

INFO:ludwig.utils.print_utils:
INFO:ludwig.utils.print_utils:╒════════════════════════╕
INFO:ludwig.utils.print_utils:│ EXPERIMENT DESCRIPTION │
INFO:ludwig.utils.print_utils:╘════════════════════════╛
INFO:ludwig.utils.print_utils:
INFO:ludwig.api:╒══════════════════╤═════════════════════════════════════════════════════════════════════════════════════════╕
│ Experiment name  │ api_experiment                                                                          │
├──────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ Model name       │ run                                                                                     │
├──────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ Output directory │ /content/results/api_experiment_run_0                                                   │
├──────────────────┼─────────────────────────────────────────────────────────────────

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:ludwig.models.llm:Done.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
INFO:ludwig.utils.tokenizers:Loaded HuggingFace implementation of meta-llama/Llama-2-7b-hf tokenizer
INFO:ludwig.models.llm:==================================================
INFO:ludwig.models.llm:Trainable Parameter Summary For Fine-Tuning
INFO:ludwig.models.llm:Fine-tuning with adapter: lora
INFO:ludwig.models.llm:==================================================
INFO:ludwig.utils.print_utils:
INFO:ludwig.utils.print_utils:╒══════════╕
INFO:ludwig.utils.print_utils:│ TRAINING │
INFO:ludwig.util

trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


INFO:ludwig.trainers.trainer:Creating fresh model training run.
INFO:ludwig.trainers.trainer:Training for 20 step(s), approximately 0 epoch(s).
INFO:ludwig.trainers.trainer:Early stopping policy: 5 round(s) of evaluation, or 100 step(s), approximately 0 epoch(s).

INFO:ludwig.trainers.trainer:Starting with step 0, epoch: 0


...

INFO:ludwig.trainers.trainer:
Running evaluation for step: 20, epoch: 0


Evaluation test : 100%|██████████| 20/20 [00:08<00:00,  2.28it/s]

INFO:ludwig.trainers.trainer:Evaluation took 16.7726s

INFO:ludwig.utils.metrics_printed_table:╒═══════════════════════╤════════════╤══════════════╤════════════╕
│                       │      train │   validation │       test │
╞═══════════════════════╪════════════╪══════════════╪════════════╡
│ bleu                  │     0.0000 │       0.0000 │     0.0000 │
├───────────────────────┼────────────┼──────────────┼────────────┤
│ char_error_rate       │   404.0000 │     385.4595 │   398.1026 │
├───────────────────────┼────────────┼──────────────┼────────────┤
│ loss                  │     9.9001 │       8.1095 │     8.1076 │
├───────────────────────┼────────────┼──────────────┼────────────┤
│ next_token_perplexity │ 29587.2969 │   27873.0918 │ 27635.6758 │
├───────────────────────┼────────────┼──────────────┼────────────┤
│ perplexity            │ 31966.7871 │   31758.9434 │ 31782.7871 │
├───────────────────────┼────────────┼──────────────┼────────────┤
│ rouge1_fmeasure       │     0.00


Training: 100%|██████████| 20/20 [00:28<00:00,  1.43s/it, loss=1.08]

INFO:ludwig.utils.print_utils:
INFO:ludwig.utils.print_utils:╒═════════════════╕
INFO:ludwig.utils.print_utils:│ TRAINING REPORT │
INFO:ludwig.utils.print_utils:╘═════════════════╛
INFO:ludwig.utils.print_utils:
INFO:ludwig.api:╒══════════════════════════════╤═══════════════════╕
│ Validation feature           │ label             │
├──────────────────────────────┼───────────────────┤
│ Validation metric            │ loss              │
├──────────────────────────────┼───────────────────┤
│ Best model step              │ 20                │
├──────────────────────────────┼───────────────────┤
│ Best model epoch             │ 1                 │
├──────────────────────────────┼───────────────────┤
│ Best model's validation loss │ 8.109542846679688 │
├──────────────────────────────┼───────────────────┤
│ Best model's test loss       │ 8.107580184936523 │
╘══════════════════════════════╧═══════════════════╛
INFO:ludwig.api:
Finished: api_experiment_run
INFO:ludwig.api:Saved to: /content/re

In [ ]:
test_examples = pd.DataFrame([
   {
   "instruction": "Predict the sentiment of the tweet:I am not tryin to get a top security clearance to my children.This was typically a false news today"
}
])

predictions = model.predict(test_examples)[0]
print(type(predictions))
keysList = list(predictions.keys())
print(keysList)
for input_with_prediction in zip(test_examples['instruction'], predictions['label_response']):
  print(f"Text: {input_with_prediction[0]}")
  print(f"Generated Output: {input_with_prediction[1]}")
  print("\n\n")

INFO:ludwig.utils.tokenizers:Loaded HuggingFace implementation of meta-llama/Llama-2-7b-hf tokenizer
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/ludwig/data/preprocessing.py:1315: UserWarning: column: 'split' is required by the dataset splitter with params: {'type': 'fixed', 'column': 'split'}, but 'split' is not present in the `dataset_df` with columns: Index(['instruction'], dtype='object'). This is acceptable during serving setting where dataset splitting is irrelevant. You may see this warning if, for example, the model was trained with a configuration that used a stratified split on the target column, but for live predictions, a value for the target column is not to be provided.
  warnings.warn(


Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:ludwig.models.llm:For generating text, using: GenerationConfig {
  "max_length": 32,
  "max_new_tokens": 512,
  "temperature": 0.3
}

INFO:ludwig.models.llm:Decoded text inputs for the first example in batch: below is the text or the sentence in the form of a tweet. detect the sentiment of the tweet and give the response in the form so as to find whether it is a positive or negative tweet. in the given training examples 0 indicates negative and 1 indicates positive.

 ### instruction: predict the sentiment of the tweet:i am not tryin to get a top security clearance to my children.this was typically a false news today

 ### response:
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
INFO:ludwig.models.llm:Decoded generated outpu

Prediction: 100%|██████████| 1/1 [00:00<00:00,  1.23it/s]


INFO:ludwig.utils.tokenizers:Loaded HuggingFace implementation of meta-llama/Llama-2-7b-hf tokenizer


<class 'pandas.core.frame.DataFrame'>
['label_predictions', 'label_probabilities', 'label_response', 'label_probability']
Text: Predict the sentiment of the tweet:I am not tryin to get a top security clearance to my children.This was typically a false news today
Generated Output: ['0\n\n']





/usr/local/lib/python3.10/dist-packages/ludwig/features/feature_utils.py:102: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(sequence_probabilities))
